# NFL Capstone EDA

I will be using SQL to create dataframes of the various tables and combinations to perform EDA

## Imports

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import os
import sys

## Connect to Database and create dataframes of each table to combine later

In [37]:
conn = sqlite3.connect('NFLstats.db')

In [38]:
teamyearsummarySQL_Query = pd.read_sql_query("SELECT * FROM 'teamyearsummary'", conn)
teamyearsummary = pd.DataFrame(teamyearsummarySQL_Query)
del teamyearsummary['id']

In [39]:
weeksSQL_Query = pd.read_sql_query("SELECT * FROM 'weeks'", conn)
weeks = pd.DataFrame(weeksSQL_Query)
del weeks['id']

In [40]:
draftSQL_Query = pd.read_sql_query("SELECT * FROM 'draft'", conn)
draft = pd.DataFrame(draftSQL_Query)
#drop drafted special teams players only 78/4595 drafted players with 3 positions
draft = draft[draft.DraftPosition != 'ST']
del draft['id']

In [41]:
startersSQL_Query = pd.read_sql_query("SELECT * FROM 'starters'", conn)
starters = pd.DataFrame(startersSQL_Query)
#Removal of whitespace in 'Starting Player'
starters['StartingPlayer'] = starters['StartingPlayer'].str.strip()
del starters['id']

In [42]:
starters.head()

,Team,Year,StartingPlayer,StartingPlayerAge,StartingPosition,StartingPlayerYrs,StartingPlayerGS
0,Arizona Cardinals,2002,Jake Plummer,1974,QB,5,16
1,Arizona Cardinals,2002,Thomas Jones,1978,RB,2,9
2,Arizona Cardinals,2002,Jason McAddley,1979,WR,0,8
3,Arizona Cardinals,2002,Frank Sanders,1973,WR,7,12
4,Arizona Cardinals,2002,Steve Bush,1974,TE,5,12


## Additional features added to starters df

In [43]:
#Create column of the number of years as a starter for that team
# This cell designates the players first year
# Have to filter by player and age because when scraping data, I had to go piece-wise, and did not generate unique playerid
starters['YrsTeamStarter'] = int(0)

for player in starters['StartingPlayer'].unique():
    for age in set(starters[(starters['StartingPlayer']==player)]['StartingPlayerAge']):
        firstyear = starters[(starters['StartingPlayer']==player) & (starters['StartingPlayerAge']==age)]['Year'].idxmin()
        starters.loc[firstyear, 'YrsTeamStarter'] = int(1)

In [ ]:
#Loop through player and year, if it is player's first year, add team to list of teams and add 1 to counter
# if not first year, but team is in list, add another year as a starter to the YrsTeamStarter column
# if not first year, but team is not in team list, this is a new team, and years as starter is back to 1
for player in starters['StartingPlayer'].unique():
    for age in set(starters[(starters['StartingPlayer']==player)]['StartingPlayerAge']):
        teamyear = []
        yrscount = 0
        for year in starters[(starters['StartingPlayer']==player) & (starters['StartingPlayerAge']==age)]['Year'].sort_values().unique():
            mask = starters[(starters['StartingPlayer']==player) & (starters['StartingPlayerAge']==age) &(starters['Year']==year)].index
            if starters.loc[mask, 'YrsTeamStarter'].iloc[-1] == 1:
                yrscount = 1
                teamyear.extend(starters.loc[mask,'Team'])
                #print(player, year, teamyear, starters.loc[mask, 'YrsTeamStarter'].iloc[-1], '1')
                continue
            elif starters.loc[mask,'Team'].iloc[-1] in teamyear:
                yrscount += 1
                starters.loc[mask, 'YrsTeamStarter'] = yrscount
                teamyear.extend(starters.loc[mask,'Team'])
                #print(player, year, teamyear, starters.loc[mask, 'YrsTeamStarter'].iloc[-1], '2')
                continue
            elif starters.loc[mask,'Team'].iloc[-1] not in teamyear:
                starters.at[mask, 'YrsTeamStarter'] = 1
                teamyear.extend(starters.loc[mask,'Team'])
                #print(player, year, teamyear, starters.loc[mask, 'YrsTeamStarter'].iloc[-1], '3')
                continue
            else:
                pass

## Import data from nfldata github repository and prepare to combine with scraped data

In [ ]:
roster = pd.read_csv('https://raw.githubusercontent.com/leesharpe/nfldata/master/data/rosters.csv')
nfldatateam = pd.read_csv('https://raw.githubusercontent.com/leesharpe/nfldata/master/data/teams.csv')

In [ ]:
#remove duplicates and account for team name changes of 4 teams
#ex original SD --> San Diego Chargers new SD --> Los Angeles Chargers 
#allows for merge with my scraped data
teamdf = nfldatateam[['team', 'full']].drop_duplicates()
teamdf.iloc[31, 1] = 'Washington Football Team'
teamdf.iloc[28, 1] = 'Los Angeles Rams'
teamdf.iloc[25, 1] = 'Los Angeles Chargers'
teamdf.iloc[22, 1] = 'Las Vegas Raiders'
teamdf.set_index('team')

In [ ]:
#merge teamdf to roster to allow for merge on full team name from scraped data
roster = pd.merge(left=roster, right=teamdf, how='left', on='team')
roster.rename(columns={'full': 'Team','season':'Year', 'full_name':'StartingPlayer', 'av':'AV'}, inplace=True)
#Remove Duplicates 
roster = roster[['Year', 'StartingPlayer', 'AV', 'Team']]
#remove instances where av is na
roster = roster[roster['AV'].notna()]
roster.drop_duplicates(inplace=True)

### Joining of Approximate Value per player per year to starters
https://www.pro-football-reference.com/blog/index6b92.html?p=465

In [ ]:
starters.head()

In [ ]:
roster.head()

In [ ]:
starters = pd.merge(left=starters, right=roster, how='left', on=['StartingPlayer','Year', 'Team'])

In [ ]:
starters

## Statistics of each table

In [ ]:
teamyearsummary.describe().T

In [ ]:
weeks.describe().T

In [ ]:
teamyearsummary.hist(bins=10, figsize=(25,20))
plt.show()

In [ ]:
weeks.hist(bins=10, figsize=(25,20))
plt.show()

### What is more correlated with the play selection of pass vs. run on offense, the team, head coach, offensive coordinator or offensive scheme?

In [ ]:
def correlation_ratio(categories, measurements):
    fcat, _ = pd.factorize(categories)
    cat_num = np.max(fcat)+1
    y_avg_array = np.zeros(cat_num)
    n_array = np.zeros(cat_num)
    for i in range(0,cat_num):
        cat_measures = measurements[np.argwhere(fcat == i).flatten()]
        n_array[i] = len(cat_measures)
        y_avg_array[i] = np.average(cat_measures)
    y_total_avg = np.sum(np.multiply(y_avg_array,n_array))/np.sum(n_array)
    numerator = np.sum(np.multiply(n_array,np.power(np.subtract(y_avg_array,y_total_avg),2)))
    denominator = np.sum(np.power(np.subtract(measurements,y_total_avg),2))
    if numerator == 0:
        eta = 0.0
    else:
        eta = np.sqrt(numerator/denominator)
    return eta

In [ ]:
coachcategories = teamyearsummary['coach']

In [ ]:
measurements = teamyearsummary['Team_Pass_Att'] / teamyearsummary['Team_Rush_Att']

In [ ]:
correlation_ratio(coachcategories, measurements)

In [ ]:
offcoorcategories = teamyearsummary['offcoor']

In [ ]:
correlation_ratio(offcoorcategories, measurements)

In [ ]:
teamcategories = teamyearsummary['Team']

In [ ]:
correlation_ratio(teamcategories, measurements)

In [ ]:
offschemecategories = teamyearsummary['offscheme']

In [ ]:
correlation_ratio(offschemecategories, measurements)

### It seems that the head coach is most likely to correlate with the pass/run ratio

## Using a correlation module that incorporates categorical and numerical features, what features correlate the most?

### Article where correlation ratio function and dython packages were found here and installed from the documentation

https://towardsdatascience.com/the-search-for-categorical-correlation-a1cf7f1888c9

http://shakedzy.xyz/dython/

In [ ]:
from dython.nominal import associations
associations(teamyearsummary, figsize=(35,35))

### How similar is the correlation between the team year summary and team week breakdown?

In [ ]:
associations(weeks, figsize=(35,35))

## What is the count of position types per round from 2002-2019

In [ ]:
roundPosition = draft.groupby(['DraftRound', 'DraftPosition'])['DraftPlayer'].count().unstack().plot(figsize=(15,10))
plt.legend(loc='lower right')
plt.margins(0.02)
plt.title('Number of Position Picks per Round Since 2002')
plt.ylabel('Position Pick Count')

In [ ]:
teamgroupeddraft = draft.groupby(['Team','DraftRound', 'DraftPosition'])['DraftPlayer'].count().unstack()

In [ ]:
teamgroupeddraft

In [ ]:
fig, axes = plt.subplots(16,2,figsize=(20,70))
for idx, team in enumerate(teamgroupeddraft.index.get_level_values(0).unique()):
        ax = teamgroupeddraft.loc[team].plot(kind='barh', stacked=True, figsize=(20,70), ax=axes.flatten()[idx])
        for p in ax.patches:
            left, bottom, width, height = p.get_bbox().bounds
            if width > 0:
                ax.annotate(str(int(width)), xy=(left+width/2, bottom+height/2), ha='center', va='center')
            else:
                pass
        ax.set_title(team + ' Draft Position Count by Round since 2002')
        ax.invert_yaxis()
        #plt.ylabel('Count')
        plt.tight_layout()
        if (idx+1)%2 == 0:
            ax.legend(frameon=False, bbox_to_anchor=(-.05,.85), loc='best', borderaxespad=0.)
        else:
            ax.legend('', frameon=False)

### How many players were selected in each position by year?

In [ ]:
draft.groupby(['Year', 'DraftPosition'])['DraftPlayer'].count().unstack().plot(figsize=(15,10))
plt.legend(loc='lower right')
plt.margins(0.02)
plt.title('Number of Position Picks per Year Since 2002')
plt.ylabel('Position Pick Count')

What is the average number of years started by position, as well as the average games started by position by year

In [ ]:
starters.groupby(['Year', 'StartingPosition'])['StartingPlayerYrs', 'StartingPlayerGS'].mean().unstack().plot(figsize=(15,10))

### Confirmation with this webpage that the groupby is what I wanted - https://www.pro-football-reference.com/draft/draft-totals.htm

In [ ]:
draft.groupby(['Year', 'DraftPosition']).count()

### Grouping team, year, and Starting Position and grouping on avg. starting years and avg. starting games started

Unstacking the multiindex and then concatenating on the teamyearsummary dataframe.  

Grouping Draft position by team and year on the team selection order

Doing all of this to see if there are any larger correlations between the teamyearsummary, starter information, and then how the draft is picked by team by year

In [ ]:
starters.columns

In [ ]:
startergroup = starters.groupby(['Team', 'Year', 'StartingPosition'])['StartingPlayerYrs', 'StartingPlayerGS', 'AV'].mean().unstack()

In [ ]:
startergroup = startergroup.reset_index()

### Flattening multiindex to concat dataframes

In [ ]:
startergroup.columns = startergroup.columns.map('{0[0]}{0[1]}'.format)

In [ ]:
startergroup

## Target Dataframes
Target features- Draft Pick Position/ Draft Team Selection

1. teamyearsummary + starters + draft 
#removeAV
AVcols = ['AVDB', 'AVDE', 'AVDT', 'AVLB', 'AVOL', 'AVQB', 'AVRB', 'AVTE', 'AVWR']
teamstarterdraft
2. teamyearsummary + startersAV + draft(2006-2018 limitation of AV data)
teamstarterdraft

3. week summary + starters + draft
4. week summary + startersAV + draft(2006-2018 limitation of AV data)


Models to try
(perform lasso Regression to see if coach/def align etc are important features)

1. Regression
    1. Ridge
    2. Random Forest Regression
    3. XGBoost
2. Classification
    1. Random Forest
    2. Gradient Boosting
    3. XGBoost



In [ ]:
draftpositionselection = draft[['Team', 'Year', 'DraftPosition','DraftTeamSelection']]

### teamyearsummary + starters + draft

In [ ]:
teamyearstarter = pd.merge(teamyearsummary, startergroup, how='inner', on=['Team', 'Year'])
teamstarterdraft = pd.merge(teamyearstarter, draftpositionselection, how='inner', on=['Team', 'Year'])

### teamyearsummary + startersAV + draft (2006-2018 limitation of AV data)

In [ ]:
teamyearstarterAV = pd.merge(teamyearsummary, startergroup, how='inner', on=['Team', 'Year'])
teamstarterdraftAV = pd.merge(teamyearstarter, draftpositionselection, how='inner', on=['Team', 'Year'])

In [ ]:
teamstarterdraft.columns

### week + starters + draft

In [ ]:
weekstarter = pd.merge(weeks, startergroup, how='inner', on=['Team', 'Year'])
weekstarterdraft = pd.merge(weekstarter, draftpositionselection, how='inner', on=['Team', 'Year'])

### week + startersAV + draft(2006-2018 limitation of AV data)

In [ ]:
weekstarterAV = pd.merge(weeks, startergroup, how='inner', on=['Team', 'Year'])
weekstarterdraftAV = pd.merge(weekstarterAV, draftpositionselection, how='inner', on=['Team', 'Year'])

## Are there any glaring correlations with our target features?

In [ ]:
from dython.nominal import compute_associations

In [ ]:
teamstarterdraftcorrAV = compute_associations(teamstarterdraftAV, clustering=True)

In [ ]:
teamstarterdraftcorr

In [ ]:
teamstarterdraftcorrAV[['DraftTeamSelection', 'DraftPosition']].sort_values('DraftTeamSelection', ascending=False)

In [ ]:
teamstarterdraftcorrAV[['DraftTeamSelection', 'DraftPosition']].sort_values('DraftPosition', ascending=False)

In [ ]:
teamstarterdraft.columns